# Вспомогательные действия

Импортирование библиотек

In [29]:
import torch
import torch.nn as nn
import numpy as np
import scipy
import scipy.stats as sps

Для воспроизводимости результатов

In [30]:
np.random.seed(seed=212225)

Тестовые данные

In [31]:
# распрелеление Коши для наличия выбросов
test_data = [sps.cauchy().rvs(1000) for _ in range(100)]

# Проверяем случай с массивом из одного элемента
test_data.append([0])

Тестовая функция, сравнивающая во float16 равенство результатов

In [32]:
def test_func(data, custom_softmax):
  result = True

  for test_case in data:
    test_case = torch.tensor(test_case)
    custom = np.array(custom_softmax(test_case), dtype=np.float16)
    exact = np.array(scipy.special.softmax(test_case), dtype=np.float16)
    if not np.array_equal(custom, exact):
      result = False
      break

  if result:
    print('Tensors are equal')
  else:
    print('Tensors are not equal')

# Определение softmax


Пусть $x \in \mathbb{R}^N$, тогда:
\begin{equation}
\text{softmax}(x)_i = \sigma(x)_i = \frac{e^{x_i}}{\sum\limits_{j=1}^{N}{e^{x_j}}} \tag{1}
\end{equation}

# Способы вычисления

## Наивный

- Напрямую вычисляем $\sum\limits_{k=1}^{N}{e^{x_k}}$
- Каждый элемент получаем по формуле (1)

In [33]:
def basic_softmax(data):
    denominator = 0

    for i in range(data.shape[0]):
        denominator += np.exp(data[i])

    result = np.zeros(data.shape[0])

    for i in range(data.shape[0]):
        result[i] = np.exp(data[i]) / denominator

    return result

In [34]:
test_func(test_data, basic_softmax)

Tensors are not equal


<ipython-input-33-4bf95bc1db0e>:5: RuntimeWarning: overflow encountered in exp
  denominator += np.exp(data[i])
<ipython-input-33-4bf95bc1db0e>:10: RuntimeWarning: overflow encountered in exp
  result[i] = np.exp(data[i]) / denominator


Чем он плох?
- **Из-за наличия показательных функций может случиться переполнение (что и видно в тестовом примере)**

Как исправить?
- **Использовать safe-softmax**

## Safe-softmax

$$ \text{safe_softmax}(x)_i = \sigma(x - \max\limits_{k = 1}^{N}{x_k})_i = \frac{e^{x_i - \max\limits_{k = 1}^{N}{x_k}}}{\sum\limits_{j=1}^{N}{e^{x_j - \max\limits_{k = 1}^{N}{x_k}}}} \tag{2}$$

In [35]:
def safe_softmax(data):
    denominator = 0
    max_elem = float('-inf')

    for i in range(data.shape[0]):
        max_elem = data[i] if data[i] > max_elem else max_elem

    for i in range(data.shape[0]):
        denominator += np.exp(data[i] - max_elem)

    result = np.zeros(data.shape[0])
    for i in range(data.shape[0]):
        result[i] = np.exp(data[i] - max_elem) / denominator

    return result

In [36]:
test_func(test_data, safe_softmax)

Tensors are equal


Возникает новая проблема - из-за необходимости нахождения $\max\limits_{k = 1}^{N}{x_k}$ и знаменателя дроби приходится дважды проходиться по массиву


**Online-softmax исправляет эту проблему**

## Online-softmax

Теперь перед вычислением softmax-a делается всего один проход по массиву, при этом на j-м шаге прохода выполняются следующие действия:


1.   $m_j = \max(m_{j-1}, x_j)$
2.   $d_j = d_{j-1} \cdot e^{m_{j - 1} - m_{j}} +  e^{x_j - m_j}$



Вспомогательные функции для обновления коэффициентов на j-м шаге и вычисления итогового знаменателя и наибольшего числа в массиве

In [37]:
def coefficient_refresher_(element, max_element, denominator):
    max_i = max_element
    max_element = element if element > max_element else max_element
    denominator = denominator * np.exp(max_i - max_element) + np.exp(element - max_element)

    return max_element, denominator

def online_softmax_coeff_values(data):
    denominator = 0
    max_element = float('-inf')

    for i in range(data.shape[0]):
      max_element, denominator = coefficient_refresher_(data[i], max_element, denominator)

    return max_element, denominator

Функция для вычисления online-softmax'а

In [38]:
def online_softmax(data):
  result = np.zeros(data.shape[0])
  m, d = online_softmax_coeff_values(data)

  for i in range(data.shape[0]):
    result[i] = np.exp(data[i] - m) / d

  return result

In [39]:
test_func(test_data, online_softmax)

Tensors are equal


Возникает вопрос - а можно ли распарралелить этот алгоритм?

Ответ - **да**

### Параллельный online-softmax

Для этого вводится ассоциаативная и коммутативная операция $\oplus$:

$ \begin{bmatrix}
m_i \\
d_i
\end{bmatrix}  $
$\oplus$
$ \begin{bmatrix}
m_j \\
d_j
\end{bmatrix}  $
=
$ \begin{bmatrix}
\max(m_i, m_j) \\
d_i \cdot e^{m_i - \max(m_i, m_j)} + d_j \cdot e^{m_j - \max(m_i, m_j)}  
\end{bmatrix}  $

Введем функцию для представленной выше операции

In [40]:
def coefficient_refresher_parallel_(m1, d1, m2, d2):
    m12 = m1 if m1 > m2 else m2
    d12 = d1 * np.exp(m1 - m12) + d2 * np.exp(m2 - m12)
    return m12, d12

Строго говоря, в этом коде нет реализации параллелизма, однако, в строке 3 части массива перемешивааются, а в строках 6-8 части исходного массива обрабатываются полностью независимо

In [41]:
def online_softmax_parallel(data, partition_size=4):
    split_size = partition_size if len(data) > partition_size else len(data)
    splits = np.array_split(data, partition_size)
    np.random.shuffle(splits)
    m, d = online_softmax_coeff_values(splits[0])

    for index in range(1, partition_size):
        new_m, new_d = online_softmax_coeff_values(splits[index])
        m, d = coefficient_refresher_parallel_(m, d, new_m, new_d)

    result = np.zeros(data.shape[0])
    for i in range(data.shape[0]):
        result[i] = np.exp(data[i] - m) / d
    return result

In [42]:
test_func(test_data, online_softmax_parallel)

Tensors are equal


# Имплементация online-softmax с помощью pytorch

In [43]:
class Online_softmax(nn.Module):
    def __init__(self):
        super(Online_softmax, self).__init__()

    def forward(self, input):
      denominator = 0
      max_elem = float('-inf')

      for i in range(input.shape[0]):
          max_i = max_elem
          max_elem = input[i] if input[i] > max_elem else max_elem
          denominator = torch.mul(denominator, torch.exp(max_i - max_elem)) + torch.exp(input[i] - max_elem)

      result = torch.exp(input - max_elem) / denominator

      return result

In [44]:
online_ = Online_softmax()
test_func(test_data, online_.forward)

Tensors are equal
